# World Bank - Commodity Prices (Pink Sheet)

World Bank Commodity Markets ("Pink Sheet") data provides monthly commodity price data.

Data source: https://www.worldbank.org/en/research/commodity-markets

API Documentation: https://datahelpdesk.worldbank.org/knowledgebase/topics/125589

## Python setup

In [1]:
# system imports
from functools import cache

In [2]:
# analytic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from readabs import recalibrate

# Note: pandas requires openpyxl to read Excel files
# Install if needed: pip install openpyxl

In [3]:
# local imports
import mgplot as mg

In [4]:
# plotting setup
CHART_DIR = "./CHARTS/WB-COMMODITIES/"
mg.set_chart_dir(CHART_DIR)
mg.clear_chart_dir()
SHOW = False
SOURCE = "World Bank Commodity Markets (Pink Sheet). "
METHOD = "** = methodology changes. "

## RBA Exchange rates

In [5]:
@cache
def usd_exchange_rate()-> pd.Series:
    """USD/AUD exchange rate from RBA"""

    hist_url = "https://www.rba.gov.au/statistics/tables/xls-hist/f11hist-1969-2009.xls"
    now_url = "https://www.rba.gov.au/statistics/tables/xls-hist/f11hist.xls"

    container = []
    for url in [hist_url, now_url]:
        table = pd.read_excel(url, sheet_name="Data", header=10, index_col=0)
        series: pd.Series = table["FXRUSD"].dropna()
        series.index = pd.PeriodIndex(series.index, freq="M")
        container.append(series)

    aud_usd = pd.concat(container)
    print("latest AUD/USD exchange rate:", aud_usd.index[-1])

    return pd.Series(aud_usd)

## World Bank Data Configuration

World Bank distributes commodity price data via Excel files (not API)

In [6]:
@cache
def get_commodity_data() -> tuple[pd.DataFrame, pd.Series]:
    """Download and process World Bank commodity price data."""

    # WARNING: 
    # This next URL may change in the future as new data is added.
    url = (
        "https://thedocs.worldbank.org/en/doc/" +
        "18675f1d1639c7a34d463f59263ba0a2-0050012025/related/CMO-Historical-Data-Monthly.xlsx"
    )
    commodities: pd.DataFrame = pd.read_excel(
        url, sheet_name="Monthly Prices", header=None, na_values=['N/A', 'missing', '-', '…', '...'], index_col=0
    )
    labels = commodities.iloc[4]
    units = commodities.iloc[5].str.replace("$", "US$").str.replace("(", "").str.replace(")", "")
    units.index = labels
    commodities.columns = labels
    commodities = commodities.iloc[7:].dropna(axis = 0, how = 'all')
    commodities.index = pd.PeriodIndex(commodities.index.str.replace("M", "-"), freq="M")
    print("latest commodity data:", commodities.index[-1])

    return commodities, units

## Plot in USD then AUD

In [7]:
def plot_commodities() -> None:

    aud = usd_exchange_rate()
    commodities, units = get_commodity_data()
    recent = 125 # months - approx 10 years

    for column in commodities.columns:
        print(column)
        series, unit = commodities[column].dropna().astype(float), units[column]
        source = SOURCE + METHOD if "**" in column else SOURCE
        mg.multi_start(
            starts=[0, -recent],
            data=series,
            function=mg.line_plot_finalise,
            title=f"{column} (USD)",
            y0=True,
            ylabel=unit,
            lfooter=source,
            annotate=True,
            show=SHOW,
        )
        if "index" in column.lower():
            continue  # skip index series
        series, unit = (series / aud).dropna(), unit.replace("US$", "AU$")
        mg.multi_start(
            starts=[0, -recent],
            function=mg.line_plot_finalise,
            data=series,
            title=f"{column} (AUD)",
            ylabel=unit,
            y0=True,
            lfooter=f"{source} RBA USD/AUD exchange rate applied. ",
            color='darkblue', 
            annotate=True,
            show=SHOW,
        )


plot_commodities()

latest AUD/USD exchange rate: 2026-01
latest commodity data: 2025-12
Crude oil, average
Crude oil, Brent
Crude oil, Dubai
Crude oil, WTI
Coal, Australian
Coal, South African **
Natural gas, US
Natural gas, Europe
Liquefied natural gas, Japan
Natural gas index
Cocoa
Coffee, Arabica
Coffee, Robusta
Tea, avg 3 auctions
Tea, Colombo
Tea, Kolkata
Tea, Mombasa
Coconut oil
Groundnuts
Fish meal
Groundnut oil **
Palm oil
Palm kernel oil
Soybeans
Soybean oil
Soybean meal
Rapeseed oil
Sunflower oil
Barley
Maize
Sorghum
Rice, Thai 5% 
Rice, Thai 25% 
Rice, Thai A.1
Rice, Viet Namese 5%
Wheat, US SRW
Wheat, US HRW
Banana, Europe
Banana, US
Orange
Beef **
Chicken **
Lamb **
Shrimps, Mexican
Sugar, EU
Sugar, US
Sugar, world
Tobacco, US import u.v.
Logs, Cameroon
Logs, Malaysian
Sawnwood, Cameroon
Sawnwood, Malaysian
Plywood
Cotton, A Index
Rubber, TSR20 **
Rubber, RSS3
Phosphate rock
DAP
TSP
Urea 
Potassium chloride **
Aluminum
Iron ore, cfr spot
Copper
Lead
Tin
Nickel
Zinc
Gold
Platinum
Silver


## The End

In [8]:
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2026-02-22 13:20:15

Python implementation: CPython
Python version       : 3.14.0
IPython version      : 9.9.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 25.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

IPython   : 9.9.0
matplotlib: 3.10.8
mgplot    : 0.2.18
numpy     : 2.4.0
pandas    : 2.3.3
readabs   : 0.1.8

Watermark: 2.6.0

